## CNN sentence classifier - Round 2 of Hyperparameter tuning

Following the analysis of round 1, this is a fine tuning of the hyperparameters.

Note that data load, model creation and training have been broken out into jbyrne_utils.py to simplify the notebook.

In [1]:
## Enable Tensorboard
%load_ext tensorboard


## Usual Imports
import numpy as np

## Add local module folder to the path and load modules

import sys
sys.path.insert(0, '../python')
import debug
from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model



In [2]:
### The following uses compatibility mode to allow use
### of CUDA 11.2 with GPU support

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# maximum number of tokens to look at.
max_len = 100


### Read in the data set of sentences into numpy array.

In [4]:
d = load_data("../data/3xNCS.json")


Loaded 11056 data records.


## Split into train, validation and test sets

Before separating inputs and labels, duplicate the positive cases twice in the training data set.

In [5]:
%%time 

train_len = int(0.8 * len(d))
val_len = int(0.2 * len(d))

train, val = np.split(d, [train_len])

pos_samples = np.array( [ i for i in d if i["label"] == 1 ])

train = np.concatenate( (train, pos_samples, pos_samples) )

# Re-randomize the training array with the additional entries
np.random.seed(42)
np.random.shuffle(train)


# split into x and y sets for use in keras runs
x_train, y_train = np.array( [ i["text"] for i in train ] ), np.array( [ i["label"] for i in train ] )
x_val,   y_val   = np.array( [ i["text"] for i in val ] ),   np.array( [ i["label"] for i in val ]   )
# x_test,  y_test  = np.array( [ i["text"] for i in test ] ),  np.array( [ i["label"] for i in test ]  )


CPU times: user 9.1 ms, sys: 11.5 ms, total: 20.6 ms
Wall time: 19.7 ms


## Tokenize the training data set and build a vocabulary from it
Want to do this to minimize the vocab to just what is in the training set for performance.

using defaults:  strip all punctuation except apostrophies, set to lower case and split on spaces. No restriction on length of vocabulary.

In [6]:
# Tokenize the training data set and build a vocabulary from it
# Want to do this to minimize the vocab to just what is in
# the training set for performance.

x_train_ids, x_val_ids, x_test_ids, tokenizer = tokenize_sentences(x_train, x_val, np.array([]), max_len)


In [7]:
# Check for % of statements that are checkable claims to 
# ensure we have some level of balance that will avoid
# high accuracy for single value guessing.

print(f"proportion of checkable claims in training data  : {np.count_nonzero(y_train == 1)/len(y_train):.4f}")
print(f"proportion of checkable claims in validation data: {np.count_nonzero(y_val == 1)/len(y_val):.4f}")
# print(f"proportion of checkable claims in test data      : {np.count_nonzero(y_test == 1)/len(y_test):.4f}")

proportion of checkable claims in training data  : 0.5395
proportion of checkable claims in validation data: 0.2432


In [8]:
%%time


# number of dimensions in the embeddings
embed_dim = 50

# Get the embedding matrix for the Embedding layer in Keras.
vocab_list, embedding_matrix, vocab_size = embed_matrix(tokenizer, embed_dim)

CPU times: user 31.1 s, sys: 78.3 ms, total: 31.2 s
Wall time: 34.9 s


In [9]:
len( embedding_matrix)

8196

In [10]:
embedding_matrix.shape[1]

50

In [11]:
print(len(x_train_ids))
print(len(y_train))
print(len(x_val_ids))
print(len(y_val))

14372
14372
2212
2212


In [12]:
# from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model

# run_model(embedding_matrix, x_train_ids, y_train, x_val_ids, y_val)

## Output of the tests are deleted as the notebook will not load in Chrome

In [13]:

# this is very quick, so we can use grid search on these models.

run_logfile = "runs2bal.log"

epochs = 20  # Not a variable, so we can determine the best epoch for the best model

#Write the header line
with open(run_logfile,'a') as f:
   f.write("timestamp|max_len|epochs|batch_size|embed_dim|num_filters|kernel_sizes|dense_layer_dims|dropout_rate|")
   for metric in [ 'loss', 'accuracy', 'val_loss', 'val_accuracy']:
       f.write(f"{metric}|")
       for i in range(0,epochs):
           f.write(f"{metric}-{i}|")
   f.write(f"END\n")        
   f.close()

    
# run the models
for k_sizes in [ [8,16], [16,32], [32,64], [4,8,12], [4,8,16], [8,12,16], [8,16,32], [4,8,16,32], [4,8,32,64] ]:

    # Set filter counts according to the number of kernels in use.
    if len(k_sizes) == 2:
        filter_counts = [ [16,16], [16,32], [32,32] ]
    if len(k_sizes) == 3:
        filter_counts = [ [16,16,16], [32, 32, 32], [64, 64, 64], [96.96 ,96] ]
    if len(k_sizes) == 4:
        filter_counts = [ [2,2,2,2], [4,4,4,4], [8,8,8,8], [32,32,32,32], [128, 128, 128, 128], [8,16,32,64] ]
    
    
    for n_fil in filter_counts:
        
        for dld in [ [8], [16], [32], [32, 16] ]:

            run_model(embedding_matrix,
                      x_train_ids,
                      y_train,
                      x_val_ids,
                      y_val,
                      kernel_sizes     = k_sizes,
                      num_filters      = n_fil,
                      dense_layer_dims = dld,
                      epochs           = epochs,
                      logfile          = run_logfile,
                      logtag           = "tblogs-bal",
                      model_dir        = "../models/CNNBal")
            print("\n\n\n##########################################################################\n\n\n")



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
288/288 [==============================] - 6s 8ms/step - loss: 0.6750 - accuracy: 0.5990 - val_loss: 0.5498 - val_accuracy: 0.7184
INFO:tensorflow:Assets written to: ../models/CNNBal/210407-154238/assets
Epoch 2/20
288/288 [==============================] - 1s 5ms/step - loss: 0.4630 - accuracy: 0.7858 - val_loss: 0.3376 - val_accuracy: 0.8562
INFO:tensorflow:Assets written to: ../models/CNNBal/210407-154238/assets
Epoch 3/20
288/288 [==============================] - 2s 6ms/step - loss: 0.3148 - accuracy: 0.8642 - val_loss: 0.2878 - val_accuracy: 0.8888
INFO:tensorflow:Assets written to: ../models/CNNBal/210407-154238/assets
Epoch 4/20
288/288 [==============================] - 2s 6ms/step - loss: 0.1979 - accuracy: 0.9256 - val_loss: 0.2152 - val_accuracy: 0.9213
INFO:tensorflow:Assets written to: ../models/CNNBal/210407-154238/assets
Epoch 5/20
288/288 [=============

288/288 [==============================] - 2s 6ms/step - loss: 0.0488 - accuracy: 0.9840 - val_loss: 0.2625 - val_accuracy: 0.9286
Epoch 8/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0289 - accuracy: 0.9908 - val_loss: 0.2333 - val_accuracy: 0.9385
Epoch 9/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0272 - accuracy: 0.9925 - val_loss: 0.3914 - val_accuracy: 0.9091
Epoch 10/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0230 - accuracy: 0.9926 - val_loss: 0.3221 - val_accuracy: 0.9277
Epoch 11/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0214 - accuracy: 0.9932 - val_loss: 0.4910 - val_accuracy: 0.9055
Epoch 12/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0181 - accuracy: 0.9941 - val_loss: 0.3346 - val_accuracy: 0.9322
Epoch 13/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0189 - accuracy: 0.9934 - val_loss: 0.5338 - val_accuracy: 0.9078
Epoch 14/20
2

Epoch 17/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0108 - accuracy: 0.9964 - val_loss: 0.5635 - val_accuracy: 0.8978
Epoch 18/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0090 - accuracy: 0.9974 - val_loss: 0.2764 - val_accuracy: 0.9566
Epoch 19/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0115 - accuracy: 0.9967 - val_loss: 0.4241 - val_accuracy: 0.9295
Epoch 20/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0094 - accuracy: 0.9975 - val_loss: 0.7918 - val_accuracy: 0.8662



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 2s 7ms/step - loss: 0.6634 - accuracy: 0.6337 - val_loss: 0.5030 - val_accuracy: 0.7608
INFO:tensorflow:Assets written to: ../models/CNNBal/210407-154544/assets
Epoch 2/20
288/288 [=====

INFO:tensorflow:Assets written to: ../models/CNNBal/210407-154656/assets
Epoch 5/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0863 - accuracy: 0.9708 - val_loss: 0.2635 - val_accuracy: 0.9150
Epoch 6/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0450 - accuracy: 0.9858 - val_loss: 0.1989 - val_accuracy: 0.9430
Epoch 7/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0367 - accuracy: 0.9880 - val_loss: 0.3145 - val_accuracy: 0.9195
Epoch 8/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0289 - accuracy: 0.9908 - val_loss: 0.4253 - val_accuracy: 0.9069
Epoch 9/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0217 - accuracy: 0.9926 - val_loss: 0.4594 - val_accuracy: 0.9064
Epoch 10/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0216 - accuracy: 0.9934 - val_loss: 0.4802 - val_accuracy: 0.9073
Epoch 11/20
288/288 [==============================] - 2s 6ms/step - loss:

288/288 [==============================] - 2s 6ms/step - loss: 0.0098 - accuracy: 0.9968 - val_loss: 0.3074 - val_accuracy: 0.9439
Epoch 15/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0120 - accuracy: 0.9959 - val_loss: 0.6480 - val_accuracy: 0.8870
Epoch 16/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0129 - accuracy: 0.9953 - val_loss: 0.4209 - val_accuracy: 0.9259
Epoch 17/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0117 - accuracy: 0.9968 - val_loss: 0.4597 - val_accuracy: 0.9272
Epoch 18/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0207 - accuracy: 0.9933 - val_loss: 0.3420 - val_accuracy: 0.9417
Epoch 19/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0096 - accuracy: 0.9972 - val_loss: 0.3642 - val_accuracy: 0.9376
Epoch 20/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0057 - accuracy: 0.9984 - val_loss: 0.4239 - val_accuracy: 0.9304



########

...

Epoch 7/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0276 - accuracy: 0.9925 - val_loss: 0.2566 - val_accuracy: 0.9195
Epoch 8/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0190 - accuracy: 0.9952 - val_loss: 0.2368 - val_accuracy: 0.9304
Epoch 9/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0109 - accuracy: 0.9976 - val_loss: 0.3723 - val_accuracy: 0.9123
Epoch 10/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0072 - accuracy: 0.9988 - val_loss: 0.3420 - val_accuracy: 0.9191
Epoch 11/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0060 - accuracy: 0.9987 - val_loss: 0.3966 - val_accuracy: 0.9127
Epoch 12/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0065 - accuracy: 0.9990 - val_loss: 0.2917 - val_accuracy: 0.9430
Epoch 13/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0062 - accuracy: 0.9983 - val_loss: 0.3024 - val_accuracy: 0.9354
Ep

Epoch 17/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0088 - accuracy: 0.9973 - val_loss: 0.4535 - val_accuracy: 0.9241
Epoch 18/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0096 - accuracy: 0.9964 - val_loss: 0.7014 - val_accuracy: 0.8987
Epoch 19/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0068 - accuracy: 0.9980 - val_loss: 0.4204 - val_accuracy: 0.9250
Epoch 20/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0058 - accuracy: 0.9978 - val_loss: 0.3105 - val_accuracy: 0.9489



##########################################################################



Adding Convolution: Kernel Size: 4, Filter Count: 32
Adding Convolution: Kernel Size: 8, Filter Count: 32
Adding Convolution: Kernel Size: 32, Filter Count: 32
Adding Convolution: Kernel Size: 64, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 3s 8ms/step - loss: 0.6224 - accuracy: 0.6481 - val_loss: 0.4069 - val_accuracy

288/288 [==============================] - 2s 7ms/step - loss: 0.0626 - accuracy: 0.9830 - val_loss: 0.1904 - val_accuracy: 0.9326
Epoch 5/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0297 - accuracy: 0.9922 - val_loss: 0.1324 - val_accuracy: 0.9598
INFO:tensorflow:Assets written to: ../models/CNNBal/210407-171756/assets
Epoch 6/20
288/288 [==============================] - 2s 8ms/step - loss: 0.0130 - accuracy: 0.9979 - val_loss: 0.2149 - val_accuracy: 0.9367
Epoch 7/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0144 - accuracy: 0.9966 - val_loss: 0.1785 - val_accuracy: 0.9485
Epoch 8/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0079 - accuracy: 0.9977 - val_loss: 0.3791 - val_accuracy: 0.9146
Epoch 9/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0044 - accuracy: 0.9992 - val_loss: 0.2045 - val_accuracy: 0.9489
Epoch 10/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0068 - ac

288/288 [==============================] - 2s 7ms/step - loss: 0.0104 - accuracy: 0.9962 - val_loss: 0.4077 - val_accuracy: 0.9367
Epoch 14/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0044 - accuracy: 0.9988 - val_loss: 0.2053 - val_accuracy: 0.9611
Epoch 15/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0082 - accuracy: 0.9975 - val_loss: 0.2716 - val_accuracy: 0.9534
Epoch 16/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0042 - accuracy: 0.9986 - val_loss: 0.3768 - val_accuracy: 0.9245
Epoch 17/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0086 - accuracy: 0.9967 - val_loss: 0.5880 - val_accuracy: 0.9042
Epoch 18/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0084 - accuracy: 0.9975 - val_loss: 0.4345 - val_accuracy: 0.9227
Epoch 19/20
288/288 [==============================] - 2s 7ms/step - loss: 0.0043 - accuracy: 0.9986 - val_loss: 0.2226 - val_accuracy: 0.9602
Epoch 20/20

Adding Convolution: Kernel Size: 4, Filter Count: 8
Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 32, Filter Count: 32
Adding Convolution: Kernel Size: 64, Filter Count: 64
Epoch 1/20
288/288 [==============================] - 3s 8ms/step - loss: 0.6663 - accuracy: 0.6160 - val_loss: 0.5698 - val_accuracy: 0.7021
INFO:tensorflow:Assets written to: ../models/CNNBal/210407-172145/assets
Epoch 2/20
288/288 [==============================] - 2s 7ms/step - loss: 0.4136 - accuracy: 0.8125 - val_loss: 0.2868 - val_accuracy: 0.8825
INFO:tensorflow:Assets written to: ../models/CNNBal/210407-172145/assets
Epoch 3/20
288/288 [==============================] - 2s 7ms/step - loss: 0.2406 - accuracy: 0.9043 - val_loss: 0.2798 - val_accuracy: 0.8847
INFO:tensorflow:Assets written to: ../models/CNNBal/210407-172145/assets
Epoch 4/20
288/288 [==============================] - 2s 7ms/step - loss: 0.1406 - accuracy: 0.9481 - val_loss: 0.1929 - val_accuracy: 0.9340






### Dataset Citations

@inproceedings{arslan2020claimbuster,
    title={{A Benchmark Dataset of Check-worthy Factual Claims}},
    author={Arslan, Fatma and Hassan, Naeemul and Li, Chengkai and Tremayne, Mark },
    booktitle={14th International AAAI Conference on Web and Social Media},
    year={2020},
    organization={AAAI}
}

@article{meng2020gradient,
  title={Gradient-Based Adversarial Training on Transformer Networks for Detecting Check-Worthy Factual Claims},
  author={Meng, Kevin and Jimenez, Damian and Arslan, Fatma and Devasier, Jacob Daniel and Obembe, Daniel and Li, Chengkai},
  journal={arXiv preprint arXiv:2002.07725},
  year={2020}
}

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]


### Useful resources in building this
Using pre-trained word embeddings: https://keras.io/examples/nlp/pretrained_word_embeddings/